# **Obtención y preparación de datos**

# OD11. Selección en Dataframes

Desde un punto de vista semántico, un dataframe puede ser considerado semejante a un diccionario de series, en el que las claves son los nombres de las columnas y los valores, las columnas (que son series pandas).

## <font color='purple'>__Material adicional__</font>


Para complementar este notebook, dejamos como material adicional 2 guias tipo tutorial para selección de datos en pandas:

- [Tutorial de Certidevs](https://certidevs.com/tutorial-pandas-operaciones-de-filtrado-busqueda-y-ordenacion): Este recurso ofrece una guía completa sobre operaciones de filtrado, búsqueda y ordenación de datos en Pandas, incluyendo ejemplos prácticos para manipular DataFrames.

- [Código Python](https://www.codigopiton.com/seleccionar-filas-de-dataframe-segun-valor-de-columnas/): Este artículo se centra en cómo seleccionar filas de un DataFrame según el valor de las columnas. Se exploran métodos como loc y query, ofreciendo ejemplos claros sobre cómo utilizarlos para filtrar datos.


### <font color='purple'>Fin material adicional </font>

In [1]:
import pandas as pd
import numpy as np

In [2]:
ventas = pd.DataFrame({
    "Entradas": [41, 32, 56, 18],
    "Salidas": [17, 54, 6, 78],
    "Valoración": [66, 54, 49, 66],
    "Límite": ["No", "Sí", "No", "No"],
    "Cambio": [1.43, 1.16, -0.67, 0.77]
},
    index = ["Ene", "Feb", "Mar", "Abr"]
)
ventas

,Entradas,Salidas,Valoración,Límite,Cambio
Ene,41,17,66,No,1.43
Feb,32,54,54,Sí,1.16
Mar,56,6,49,No,-0.67
Abr,18,78,66,No,0.77


Es posible utilizar la sintaxis de los diccionarios para seleccionar la columna Entradas.

In [3]:
print(type(ventas["Entradas"]))

<class 'pandas.core.series.Series'>


In [4]:
ventas["Entradas"]

,Entradas
Ene,41
Feb,32
Mar,56
Abr,18


Esto significa que podemos realizar una selección en dicho resultado para, por ejemplo, extraer el valor correspondiente a febrero.

In [5]:
ventas["Entradas"]["Feb"]

32

Usar comas para separar no funciona y por lo tanto nos entrega un error.

In [6]:
try:
    # La separación por comas no funciona, nos entrega un error de llave
    ventas["Entradas", "Feb"]
except Exception as e:
    print(e.__doc__)
    print(type(e).__name__)


Mapping key not found.
KeyError


Si, una vez seleccionada una columna, le asignamos una lista o array (o serie) de valores de la misma longitud, estamos modificando dicha columna del dataframe.

In [7]:
ventas["Entradas"] = [33, 25, 40, 12]
ventas

,Entradas,Salidas,Valoración,Límite,Cambio
Ene,33,17,66,No,1.43
Feb,25,54,54,Sí,1.16
Mar,40,6,49,No,-0.67
Abr,12,78,66,No,0.77


Si asignamos un único valor escalar, este se propaga por toda la columna.

In [8]:
ventas["Salidas"] = 1
ventas

,Entradas,Salidas,Valoración,Límite,Cambio
Ene,33,1,66,No,1.43
Feb,25,1,54,Sí,1.16
Mar,40,1,49,No,-0.67
Abr,12,1,66,No,0.77


Si estuviésemos asignando un array cuya longitud no coincidiese con la de la columna (y no estuviésemos asignando un escalar), obtendríamos un error.

Si asignamos una serie pandas se consideran los índices del dataframe y de la serie, haciendo coincidir los valores cuyos índices sean los mismos en ambas estructuras (si dicha columna no existe, se crea). En el caso de que haya valores en la serie con índices que no se encuentren en el dataframe, se descartan. Y en el caso de que haya índices en el dataframe que no se encuentren en la serie, se asigna un valor *NaN*.

Así, en el siguiente ejemplo, estamos añadiendo una serie cuyos índices son "Ene", "Mar", "Abr" y "May". Es decir, la serie no tiene un valor para el índice "Feb" que sí se encuentra en el dataframe (se asigna un *NaN*), e incluye el índice "May" que no se encuentra en el dataframe y se descarta.

In [9]:
# Feb no existe en la serie, luego se completa con NaN
# Mar no existe en el DataFrame, luego se descarta
ventas["Perdidas"] = pd.Series([5, 4, 6, 8], index = ["Ene", "Mar", "Abr", "May"])
ventas

,Entradas,Salidas,Valoración,Límite,Cambio,Perdidas
Ene,33,1,66,No,1.43,5.0
Feb,25,1,54,Sí,1.16,NaN
Mar,40,1,49,No,-0.67,4.0
Abr,12,1,66,No,0.77,6.0


Los valores asignados pueden proceder del propio dataframe:

In [10]:
ventas["Ganancias"] = (ventas["Entradas"]*2) - (ventas["Valoración"]/10)
ventas

,Entradas,Salidas,Valoración,Límite,Cambio,Perdidas,Ganancias
Ene,33,1,66,No,1.43,5.0,59.4
Feb,25,1,54,Sí,1.16,NaN,44.6
Mar,40,1,49,No,-0.67,4.0,75.1
Abr,12,1,66,No,0.77,6.0,17.4


También podemos acceder a una columna con la llamada "notación punto".

In [11]:
ventas.Ganancias = 1
ventas

,Entradas,Salidas,Valoración,Límite,Cambio,Perdidas,Ganancias
Ene,33,1,66,No,1.43,5.0,1
Feb,25,1,54,Sí,1.16,NaN,1
Mar,40,1,49,No,-0.67,4.0,1
Abr,12,1,66,No,0.77,6.0,1


Deberemos tener en cuenta que con esta notación no es posible crear nuevas columnas ni eliminarlas y que solo funcionará si el nombre de la columna no incluye espacios en blanco y no coincide con ninguna palabra reservada de Python.

El uso de un rango numérico entre los corchetes realiza una selección de filas, lo que puede parecer una cierta incoherencia.

In [12]:
import numpy as np
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


In [13]:
df[2:5]

,A,B,C
c,6,7,8
d,9,10,11
e,12,13,14


El equipo de pandas lo justifica diciendo que esta sintaxis resulta extremadamente conveniente al tratarse de un tipo de selección frecuentemente usada. Esto es cierto, pero el hecho de que selecciones aparentemente semejantes (`df[1,2]`, `df[[1, 2]]`, `df[1:3, 5]`, etc.) devuelvan un error no facilita su comprensión.

Se devuelven las filas entre el primer valor del rango (incluido) y el último (sin incluir). También podríamos haber usado las etiquetas del índice.

In [14]:
df["b":"d"]

,A,B,C
b,3,4,5
c,6,7,8
d,9,10,11


In [15]:
df[:3]

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8


In [16]:
df[:"c"]

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8


Al situar entre los corchetes una __lista de etiquetas__, estaremos seleccionando columnas en el orden en el que aparecen en la lista y con formato dataframe:

In [17]:
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


In [18]:
# Noten los dobles corchetes. El interior representa una lista
print(type(df[["C", "A"]]))
df[["C", "A"]]

<class 'pandas.core.frame.DataFrame'>


,C,A
a,2,0
b,5,3
c,8,6
d,11,9
e,14,12
f,17,15


También es posible extraer de forma segura una columna de un dataframe usando el método `pandas.DataFrame.get`. Éste extrae la columna indicada devolviendo un valor alternativo (por defecto `None`) si dicha columna no existe.

In [19]:
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df.get("A")

,A
a,0
b,3
c,6
d,9
e,12
f,15


In [20]:
df.get("D")

Al igual que ocurre con las series, el método `pandas.DataFrame.loc` permite seleccionar un conjunto de filas y columnas por etiquetas. Este método acepta diferentes argumentos.

In [21]:
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


In [22]:
df.loc["c"]

,c
A,6
B,7
C,8


El resultado es una serie pandas con las etiquetas de columnas del dataframe original como índice.

Es necesario mencionar que el argumento será siempre interpretado como etiqueta, aun cuando pueda estar representando un índice válido.

In [23]:
df = pd.DataFrame(np.arange(12).reshape([4, 3]),
                  index = [1, 3, 0, 4],
                  columns = ["A", "B", "C"])
df

,A,B,C
1,0,1,2
3,3,4,5
0,6,7,8
4,9,10,11


In [24]:
# No es el índice cero, es la etiqueta 0
df.loc[0]

,0
A,6
B,7
C,8


Si la etiqueta no existe, se devuelve un error (nuevamente, aun cuando la etiqueta sea un número que pueda estar representando un índice válido).

In [25]:
try:
    df.loc[5]
except Exception as e:
    print(e.__doc__)
    print(type(e).__name__)

Mapping key not found.
KeyError


Si pasamos a `loc` una __lista de etiquetas__, estaremos extrayendo las filas cuyas etiquetas se indican, y en el orden en el que aparezcan en la lista.

In [26]:
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


In [27]:
df.loc[["c", "a", "e"]]

,A,B,C
c,6,7,8
a,0,1,2
e,12,13,14


Al contrario de lo que ocurre cuando solo indicamos una etiqueta, el resultado es un dataframe. Y lo es aún cuando la lista contenga un único elemento.

In [28]:
print(type(df.loc[["c"]]))
df.loc[["c"]]

<class 'pandas.core.frame.DataFrame'>


,A,B,C
c,6,7,8


In [29]:
print(type(df.loc["c"]))
df.loc["c"]

<class 'pandas.core.series.Series'>


,c
A,6
B,7
C,8


Otra opción es utilizar rangos limitados por etiquetas.

In [30]:
df.loc["b":"d"]

,A,B,C
b,3,4,5
c,6,7,8
d,9,10,11


Obsérvese que la selección incluye todas las filas incluyendo las dos de los extremos del rango.

En los ejemplos vistos hasta ahora estamos extrayendo una o varias filas para todas las columnas. En posible, por supuesto, especificar qué filas y qué columnas exactas queremos extraer. Así, si utilizamos una única etiqueta para indicar la fila, y una única etiqueta para indicar la columna, separadas por una coma, estaremos extrayendo un único valor.

In [31]:
df.loc["a", "C"]

2

Podemos sustituir una de las dos etiquetas por el símbolo de dos puntos (:), lo que supondrá seleccionar todos los elementos de ese eje.

In [32]:
df.loc[:, "A"]

,A
a,0
b,3
c,6
d,9
e,12
f,15


Esto supone que, por ejemplo, las dos expresiones siguientes devuelven el mismo resultado:

In [33]:
df.loc["b"]

,b
A,3
B,4
C,5


In [34]:
df.loc["b",:]

,b
A,3
B,4
C,5


Los métodos vistos pueden combinarse. Podemos, por ejemplo, seleccionar la intersección de las filas $e$ y $c$ (en este orden) y la columna $B$.

In [35]:
df.loc[["e", "c"], "B"]

,B
e,13
c,7


El método `pandas.DataFrame.iloc` permite realizar selecciones por posición. Tal y como cabría esperar, pueden utilizarse diferentes tipos de argumentos que determinan qué elementos se están extrayendo.

In [36]:
df = pd.DataFrame(np.random.randint(0, 10, 18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,8,7,2
b,2,5,6
c,0,8,8
d,5,8,8
e,5,0,6
f,8,2,8


In [37]:
df.iloc[2]

,c
A,0
B,8
C,8


El número indicado siempre será tratado como posición, y no como etiqueta.

In [38]:
df = pd.DataFrame(np.arange(12).reshape([4, 3]),
                  index = [3, 2, 1, 0],
                  columns = ["A", "B", "C"])
df

,A,B,C
3,0,1,2
2,3,4,5
1,6,7,8
0,9,10,11


In [39]:
df.iloc[3]

,0
A,9
B,10
C,11


Si el número es negativo, hace referencia al final del dataframe.



In [40]:
df.iloc[-1]

,0
A,9
B,10
C,11


Si utilizamos como argumento una __lista o array de números__, estamos extrayendo las filas cuyos índices son los elementos del mismo, y en el orden en el que aparecen en él.

In [41]:
df = pd.DataFrame(np.random.randint(0, 10, 18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,7,2,1
b,8,3,4
c,3,8,9
d,6,4,8
e,9,2,4
f,4,9,5


In [42]:
# Noten los dobles corchetes
df.iloc[[3, 1]]

,A,B,C
d,6,4,8
b,8,3,4


Si utilizamos un rango de números, como en el siguiente ejemplo en el que indicamos como argumento 2:4, estamos extrayendo las filas cuyos índices van de la primera cifra del rango incluida (2 en el ejemplo) hasta la última cifra sin incluir (4 en el ejemplo).

In [43]:
df.iloc[2:4]

,A,B,C
c,3,8,9
d,6,4,8


Como suele ser habitual, si no se especifica el primer valor, se consideran las filas desde la primera. Y si no se especifica el último valor, se consideran las filas hasta la última (incluida).

In [44]:
df.iloc[:3]

,A,B,C
a,7,2,1
b,8,3,4
c,3,8,9


In [45]:
df.iloc[4:]

,A,B,C
e,9,2,4
f,4,9,5


También pueden usarse valores negativos para especificar el comienzo o el final del rango.

Si añadimos un segundo argumento, estaremos haciendo referencia al índice de columna.

In [46]:
df.iloc[3, 1]

4

In [47]:
df.iloc[2:4, [1,2]]

,B,C
c,8,9
d,4,8


En ocasiones nos encontraremos con que resultaría de utilidad poder realizar selecciones mezclando etiquetas e índices, y los métodos vistos, `loc` e `iloc`, solo permiten el uso de etiquetas o de índices, respectivamente. Para poder mezclar ambos tipos de referencias podemos recurrir a los métodos `pandas.Index.get_loc` y `pandas.Index.get_indexer`, métodos asociados a los índices de un dataframe.

El primero, `get_loc`, devuelve el índice de la etiqueta que se adjunte como parámetro. El segundo, `get_indexer`, devuelve un array con los índices de las etiquetas que se adjunten en forma de lista como parámetro.

In [48]:
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


Si aplicamos estos métodos al índice de columnas haciendo referencia a etiquetas de columnas, obtenemos los siguientes resultados:

In [49]:
df.columns.get_loc("B")

1

In [50]:
df.columns.get_indexer(["A", "C"])

array([0, 2])

En el primer caso hemos pasado la etiqueta "B" y el método ha devuelto su índice (1). En el segundo caso hemos pasado una lista de etiquetas y hemos obtenido un array con sus índices.

Si ejecutamos estos métodos en el índice de filas:

In [51]:
df.index.get_loc("d")

3

In [52]:
df.index.get_indexer(["c", "e"])

array([2, 4])

Ahora que sabemos cómo convertir etiquetas en sus índices equivalentes, podemos seleccionar datos de un dataframe mezclando etiquetas e índices si convertimos las etiquetas y utilizamos el método `iloc` ya visto. Por ejemplo, si quisiéramos extraer del anterior dataframe el dato que ocupa la fila $c$ y la columna de índice 2, podríamos conseguirlo del siguiente modo:

In [53]:
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


In [54]:
df.iloc[df.index.get_loc("c"), 2]

8

O si deseásemos obtener de las filas 5 y 3 (en este orden) los valores correspondientes a las columnas $C$ y $A$ (en este orden), podríamos hacerlo con la siguiente expresión:

In [55]:
df.iloc[[5, 3], df.columns.get_indexer(["C", "A"])]

,C,A
f,17,15
d,11,9


Otro método especialmente útil para la selección es el uso de listas de booleanos. Nuevamente puede parecer un tanto incoherente aunque, en este caso, su uso sí es extremadamente conveniente.

Si partimos del mismo dataframe usado en la sección anterior, podemos crear una lista de booleanos (que, por motivos puramente pedagógicos, asignamos a una variable, *mask*) y realizar la selección con ella entre los corchetes. Vemos a continuación que este método también selecciona filas del dataframe.

In [56]:
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
mask = [True, False, True, False, False, True]
df[mask]

,A,B,C
a,0,1,2
c,6,7,8
f,15,16,17


El vector de booleanos deberá tener la misma longitud que el índice de filas (es decir, un booleano por fila) y la selección devolverá aquellas filas para las que el elemento correspondiente del vector tome el valor `True`.

La verdadera potencia de este estilo de selección se pone de manifiesto cuando la máscara se genera a partir de los datos del propio dataframe. Por ejemplo, si queremos seleccionar las filas para las que el valor de la columna $A$ sea mayor que 7:

In [57]:
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


In [58]:
df[df.A > 7]

,A,B,C
d,9,10,11
e,12,13,14
f,15,16,17


Este tipo de filtrados resultan muy frecuentes en entornos de análisis, de ahí que la posibilidad de realizarlos sin necesidad de recurrir a métodos adicionales (`loc`, `iloc` o `get`, por ejemplo) resulte tan conveniente.

Con `loc` podemos usar directamente una expresión de comparación como la vista:

In [59]:
df.loc[df.B > 6]

,A,B,C
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


Sin embargo, con `iloc` nos veremos obligados a extraer los valores del dataframe resultante de la comparación -tal y como ocurría con las series- pues, de otro modo, obtendremos un error. Recordar que no hacepta series como argumentos, solo arreglos o listas.

In [60]:
df.iloc[(df.B > 6).values]

,A,B,C
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


In [61]:
df.iloc[list(df.B > 6)]

,A,B,C
c,6,7,8
d,9,10,11
e,12,13,14
f,15,16,17


**Evitamos problemas si, tal y como sugiere pandas, utilizamos siempre el método loc.**

Al igual que ocurre con las series, también los dataframes tienen un método que permite extraer elementos del mismo de forma aleatoria: `pandas.DataFrame.sample`. Este método permite especificar el número de elementos a extraer (o el porcentaje respecto del total, parámetros `n` y `frac`, respectivamente), si la extracción se realiza con reemplazo o no (parámetro `replace`), los pesos a aplicar a los elementos para realizar una extracción aleatoria ponderada (parámetro `weights`) y una semilla para el generador de números aleatorios que asegure la reproducibilidad de la extracción (parámetro `random_state`). También es posible indicar el eje a lo largo del cual se desea realizar la extracción (por defecto se extraen filas, correspondiente al eje 0).

In [62]:
df = pd.DataFrame(np.arange(18).reshape([6, 3]),
                  index = ["a", "b", "c", "d", "e", "f"],
                  columns = ["A", "B", "C"])
df.sample(3, random_state = 18)

,A,B,C
f,15,16,17
e,12,13,14
b,3,4,5


Si especificamos como eje el valor 1, estaremos extrayendo columnas.

In [63]:
df.sample(2, random_state = 18, axis = 1)

,A,B
a,0,1
b,3,4
c,6,7
d,9,10
e,12,13
f,15,16


Si hacemos uso del parámetro `frac`, podemos especificar el porcentaje de elementos a extraer.

In [64]:
df.sample(frac = 0.6, random_state = 18)

,A,B,C
f,15,16,17
e,12,13,14
b,3,4,5
a,0,1,2


Otra forma de extraer datos es la proporcionada por el método `pandas.DataFrame.pop`, que extrae y elimina una columna de un dataframe.

In [65]:
df = pd.DataFrame(np.arange(15).reshape([3, 5]),
                  index = ["a", "b", "c"],
                  columns = ["A", "B", "C", "D", "E"])
df

,A,B,C,D,E
a,0,1,2,3,4
b,5,6,7,8,9
c,10,11,12,13,14


In [66]:
columna = df.pop("B")
columna

,B
a,1
b,6
c,11


In [67]:
df

,A,C,D,E
a,0,2,3,4
b,5,7,8,9
c,10,12,13,14


## <font color='purple'> __EXPERIMENTO__: </font>

### seleccionando datos de un dataframe utilizando enmascaramiento booleano
Consideramos que el enmascaramiento booleano es una herramienta muy útil para la ciencia de datos. A continuación realizaremos 3 enmascaramientos booleanos para acceder a los datos sin editar el dataframe de origen.


In [68]:
# creación en un dataframe con valores numéricos al azar
# semilla para la reproducibilidad
np.random.seed(42)
df = pd.DataFrame(np.random.randint(1, 101, size=(1000, 10)), columns=[f'Columna_{i+1}' for i in range(10)])
df.head()

,Columna_1,Columna_2,Columna_3,Columna_4,Columna_5,Columna_6,Columna_7,Columna_8,Columna_9,Columna_10
0,52,93,15,72,61,21,83,87,75,75
1,88,100,24,3,22,53,2,88,30,38
2,2,64,60,21,33,76,58,22,89,49
3,91,59,42,92,60,80,15,62,62,47
4,62,51,55,64,3,51,7,21,73,39


Primero crearemos una mascara que permita filtrar las filas donde todos sus valores son pares. La llamaremos 'solo_pares'
- Para considerar todos los valores de la fila utilizaremos el metodo `all(axis=1)`

In [69]:
# creamos la expresión que revisa si el valor es par y le agregamos el metodo all(axis=1) para que revise en toda la fila
solo_pares = (df % 2 == 0).all(axis=1)
# aplicamos la mascara al dataframe
df[solo_pares]

,Columna_1,Columna_2,Columna_3,Columna_4,Columna_5,Columna_6,Columna_7,Columna_8,Columna_9,Columna_10
75,76,24,28,8,92,36,90,8,58,60
642,10,78,18,88,14,34,58,76,22,70


Otro ejemplo es buscar filas que contengan valores específicos en cualquier columna.
  - para ello utilizaremos una lista y el método `any(axis=1)`

In [70]:
target = [3,7,9]
mask = df.isin(target).any(axis=1)
df[mask]

,Columna_1,Columna_2,Columna_3,Columna_4,Columna_5,Columna_6,Columna_7,Columna_8,Columna_9,Columna_10
1,88,100,24,3,22,53,2,88,30,38
4,62,51,55,64,3,51,7,21,73,39
5,18,4,89,60,14,9,90,53,2,84
11,9,88,1,8,88,63,11,81,8,35
12,35,33,5,41,28,7,73,72,12,34
...,...,...,...,...,...,...,...,...,...,...
979,10,83,30,22,3,82,75,48,2,76
983,9,87,35,71,79,53,54,84,25,67
987,9,33,60,93,77,21,58,85,69,10
991,35,77,52,31,87,68,52,80,7,22


Ahora probemos seleccionar filas en donde la columna_3 sea mayor a 10, la columna_7 sea par y la columna_9 sea impar.
- para esto debemos crear 3 tipos de enmascaramiento booleano


In [71]:
mayor_10 = df["Columna_3"] > 10
par_7 = (df["Columna_7"] % 2 == 0)
impar_9 = (df["Columna_9"] % 2 != 0)
enmascaramiento_condiciones = mayor_10 & par_7 & impar_9
df[enmascaramiento_condiciones]



,Columna_1,Columna_2,Columna_3,Columna_4,Columna_5,Columna_6,Columna_7,Columna_8,Columna_9,Columna_10
2,2,64,60,21,33,76,58,22,89,49
9,72,78,87,62,40,85,80,82,53,24
20,9,62,37,97,51,44,24,79,59,32
25,11,16,97,73,59,70,80,93,3,20
26,59,36,19,90,67,19,20,96,71,52
...,...,...,...,...,...,...,...,...,...,...
987,9,33,60,93,77,21,58,85,69,10
988,6,13,65,20,57,46,8,10,73,67
991,35,77,52,31,87,68,52,80,7,22
992,93,75,56,97,58,93,64,12,7,55


### <font color='purple'>Fin experimento </font>

### <font color='green'>Actividad 1</font>

Escribir una función que reciba un DataFrame con el formato de la siguiente tabla:

| Mes     | Ventas | Gastos |
| ------- | -----: | -----: |
| Enero   |  30500 |  22000 |
| Febrero |  35600 |  23400 |
| Marzo   |  28300 |  18100 |
| Abril   |  33900 |  20700 |
| Mayo    |  31450 |  25620 |
| Junio   |  33040 |  25500 |

y devuelva el balance (ventas - gastos) para los siguientes periodos:

1. Enero a Marzo
2. Abril a Junio
3. Todo el periodo

In [72]:
# Tu código aquí ...
def balance(dataframe):
    dataframe["Balance"] = dataframe["Ventas"] - dataframe["Gastos"]
    balance = pd.DataFrame()
    balance.loc["Enero - Marzo", "Balance"] = dataframe["Balance"].loc[0:2].sum()  # Enero, Febrero, Marzo
    balance.loc["Abril - Junio", "Balance"] = dataframe["Balance"].loc[3:5].sum()  # Abril, Mayo, Junio
    balance.loc["Todo el periodo", "Balance"] = dataframe["Balance"].sum()  # Total
    return balance

df = pd.DataFrame({
    'Mes': ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio'],
    'Ventas': [30500, 35600, 28300, 33900, 31450, 33040],
    'Gastos': [22000, 23400, 18100, 20700, 25620, 25500]
})

# Calcular el balance
result_df = balance(df)
result_df



,Balance
Enero - Marzo,30900.0
Abril - Junio,26570.0
Todo el periodo,57470.0


<font color='green'>Fin Actividad 1</font>

### <font color='green'>Actividad 2</font>

Dado el siguiente DataFrame:

```
dates = pd.date_range('20230101', periods=6)
df1 = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
```

Selecciona las filas correspondientes a las fechas después del 2023-01-03.

In [73]:
# Tu código aquí ...
dates = pd.date_range('20230101', periods=6)
df1 = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
df1.loc["2023-01-04":]

,A,B,C,D
2023-01-04,-0.692865,-0.054459,-0.437207,-1.625782
2023-01-05,-0.772415,0.889541,1.588294,0.651275
2023-01-06,1.543698,0.085389,1.742376,2.054488


<font color='green'>Fin Actividad 2</font>

### <font color='green'>Actividad 3</font>

Dado el siguiente DataFrame:

```
df2 = pd.DataFrame(np.random.randint(0, 100, size=(8, 4)), columns=list('WXYZ'))
```

Selecciona las filas de índice 2 a 5, pero solo las columnas 'W' y 'Z'.

In [74]:
# Tu código aquí ...
df2 = pd.DataFrame(np.random.randint(0, 100, size=(8, 4)), columns=list('WXYZ'))
df2.loc[2:5, ["W", "Z"]]

,W,Z
2,63,50
3,77,49
4,33,30
5,53,70


<font color='green'>Fin Actividad 3</font>

### <font color='green'>Actividad 4</font>

Dado el siguiente DataFrame:

```
data = {
    'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
    'B': ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
    'C': np.random.randint(1, 10, 8),
    'D': np.random.randint(1, 10, 8)
}
df3 = pd.DataFrame(data)
```

Selecciona todos los registros donde A sea 'foo' y B no sea 'three'.

In [75]:
# Tu código aquí ...
data = {
    'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
    'B': ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
    'C': np.random.randint(1, 10, 8),
    'D': np.random.randint(1, 10, 8)
}
df3 = pd.DataFrame(data)
df3[(df3["A"] == "foo") & (df3["B"] != "three")]

,A,B,C,D
0,foo,one,6,1
2,foo,two,2,7
4,foo,two,1,6
6,foo,one,4,2


<font color='green'>Fin Actividad 4</font>

### <font color='green'>Actividad 5</font>

Dado el siguiente DataFrame:

```
df4 = pd.DataFrame(np.random.randint(-10, 10, size=(6, 6)), columns=list('ABCDEF'))
```

Crea una máscara que identifique los valores negativos en el DataFrame y luego reemplace esos valores con su valor absoluto utilizando una función.

In [76]:
# Tu código aquí ...
df4 = pd.DataFrame(np.random.randint(-10, 10, size=(6, 6)), columns=list('ABCDEF'))
df4[df4 < 0] = df4[df4 < 0].abs()
df4

,A,B,C,D,E,F
0,6,6,3,3,1,3
1,8,8,7,8,8,4
2,5,5,3,5,7,9
3,5,0,4,6,6,1
4,2,1,4,3,10,6
5,7,6,9,4,1,6


<font color='green'>Fin Actividad 5</font>

### <font color='green'>Actividad 6</font>

Dado el siguiente DataFrame:

```
df5 = pd.DataFrame({
    'key': list('ABCDAE'),
    'values': [10, 15, 20, 25, 30, 35],
    'dates': pd.date_range(start='20230101', periods=6)
})
```

Selecciona las filas con key igual a 'A', pero solo muestra las dos primeras entradas y las columnas a partir de la segunda columna.

In [77]:
# Tu código aquí ...
df5 = pd.DataFrame({
    'key': list('ABCDAE'),
    'values': [10, 15, 20, 25, 30, 35],
    'dates': pd.date_range(start='20230101', periods=6)
})
df5[(df5["key"] == "A")].iloc[:2, 1:]

,values,dates
0,10,2023-01-01
4,30,2023-01-05


<font color='green'>Fin Actividad 6</font>

### <font color='green'>Actividad 7</font>

Dado el siguiente DataFrame:

```
df6 = pd.DataFrame({
    'region': ['North', 'South', 'West', 'East', 'Central', 'North'],
    'sales': [320, 234, 489, 278, 213, 389],
    'expenses': [245, 267, 248, 310, 190, 245]
})
```

Identifica y selecciona las regiones donde las ventas exceden los gastos por más del 20%.

In [78]:
# Tu código aquí ...
df6 = pd.DataFrame({
    'region': ['North', 'South', 'West', 'East', 'Central', 'North'],
    'sales': [320, 234, 489, 278, 213, 389],
    'expenses': [245, 267, 248, 310, 190, 245]
})
df6[(df6["sales"] / df6["expenses"]) > 1.2]

,region,sales,expenses
0,North,320,245
2,West,489,248
5,North,389,245


<font color='green'>Fin Actividad 7</font>

<img src="https://drive.google.com/uc?export=view&id=1Igtn9UXg6NGeRWsqh4hefQUjV0hmzlBv" width="100" align="left" title="Runa-perth">
<br clear="left">


##<font color='red'>**Actividad Avanzada**</font>

### <font color='green'>Actividad 8</font>

Estás analizando un conjunto de datos de ventas y existencias de una tienda a lo largo del tiempo. Tienes una Serie que representa las fechas en que se realizaron auditorías sorpresa y un DataFrame que contiene registros de ventas diarias, existencias y otros detalles relevantes.

El objetivo es inspeccionar cómo se comportaron las ventas y existencias en las fechas cercanas a las auditorías.


```
# Serie de auditorías
audit_dates = pd.Series(pd.to_datetime(['2023-01-15', '2023-03-05', '2023-06-20', '2023-08-25']))

# DataFrame de registros
dates = pd.date_range('20230101', '20230930', freq='D')
data = {
    'sales': np.random.randint(50, 200, len(dates)),
    'stock': np.random.randint(100, 500, len(dates)),
    'returns': np.random.randint(1, 50, len(dates))
}
df = pd.DataFrame(data, index=dates)
```

1. Identifica las fechas en las que se realizaron las auditorías utilizando el método isin() en el DataFrame, y muestra las ventas y existencias para esas fechas.
2. Usando iloc, selecciona los registros de 2 días antes y 2 días después de cada fecha de auditoría (es decir, una ventana de 5 días para cada auditoría).
3. Filtra este resultado para mostrar sólo aquellos días en los que las ventas fueron menores que el promedio de ventas del período y las existencias estaban por encima del promedio.
4. Utiliza el método loc para agregar una nueva columna llamada "audit_period" al DataFrame original, marcando con True los registros que caen dentro de las ventanas de auditoría y False en caso contrario.


In [79]:
# Tu código aquí ...

# Serie de auditorías
audit_dates = pd.Series(pd.to_datetime(['2023-01-15', '2023-03-05', '2023-06-20', '2023-08-25']))

# DataFrame de registros
dates = pd.date_range('20230101', '20230930', freq='D')
data = {
    'sales': np.random.randint(50, 200, len(dates)),
    'stock': np.random.randint(100, 500, len(dates)),
    'returns': np.random.randint(1, 50, len(dates))
}
df = pd.DataFrame(data, index=dates)


<font color='green'>Fin Actividad 8</font>

<img src="https://drive.google.com/uc?export=view&id=1Igtn9UXg6NGeRWsqh4hefQUjV0hmzlBv" width="50" align="left" title="Runa-perth">
<br clear="left">